In [4]:
import os
import sys

In [212]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [213]:
from functions import json_get_sales_count, json_get_fooditem_keys, \
    merge_data, modify_into_training_data

from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [214]:
import requests
import datetime

In [248]:
def get_accesstoken():
    data = {
        'grant_type': grant_type,
        'client_id': client_id_1,
        'client_secret': client_secret_1,
        'refresh_token': refresh_token_1,
        'client_version': client_version
    }
    resp = requests.post(url=token_url,
                         json=data)
    resp_json = resp.json()
    return resp_json['access_token']

accesstoken = get_accesstoken()

In [249]:
def get_summary_report(token, startdate, enddate):
    arr = {
        'searchCriteria': {
            'startDate': startdate,
            'endDate': enddate,
            'includedReports': [102]
        }
    }
    result = requests.post(url='https://mapi-eu.talech.com/reports/receiptssummaryreport',
                           json=arr,
                           headers=token)
    items = result.json()
    salesdata = json_get_sales_count(items)
    return salesdata

In [250]:
import calendar

In [251]:
from dateutil.relativedelta import relativedelta

In [252]:
def get_15month_back_dates(month, year):
    target = datetime.date(year, month, 1)
    lst = []
    
    for h in range(-15, -12, 1):
        prev_year = target + relativedelta(months=h)
        for i in range(1, calendar.monthlen(prev_year.year, prev_year.month) + 1):
            d = datetime.date(prev_year.year, prev_year.month, i)
            lst.append(d.strftime("%m/%d/%Y 00:00:00"))
    return lst

In [253]:
def get_3month_back_dates(month, year):
    target = datetime.date(year, month, 1)
    lst = []
    
    for h in range(-3, 0, 1):
        prev_year = target + relativedelta(months=h)
        for i in range(1, calendar.monthlen(prev_year.year, prev_year.month) + 1):
            d = datetime.date(prev_year.year, prev_year.month, i)
            lst.append(d.strftime("%m/%d/%Y 00:00:00"))
    return lst

In [254]:
token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [255]:
prevyear = get_15month_back_dates(2, 2019)
prevyearsales = {}
for i in range(1, len(prevyear)):
    salesdata = get_summary_report(token, prevyear[i-1], prevyear[i])
    prevyearsales[prevyear[i-1]] = salesdata

In [256]:
currentyear = get_3month_back_dates(2, 2019)
currentyearsales = {}
for i in range(1, len(currentyear)):
    salesdata = get_summary_report(token, currentyear[i-1], currentyear[i])
    currentyearsales[currentyear[i-1]] = salesdata

In [257]:
prevyearsales

{'11/01/2017 00:00:00': {'BAKED GOODS': {'ALM-FIN-TG': 5,
   'SCONE-NHTG': 12,
   'CC-BROW-SI': 3,
   'CROIS-SI': 2,
   'PUM-PIE-TG': 2,
   'SCONE-NHSI': 1,
   'SCONE-HTG': 1,
   'PUM-PIE-SI': 1,
   'QUIN-TG': 4,
   'OATCOOK-SI': 1,
   'ANZAC-TG': 2,
   'ALM-FIN-SI': 3,
   'CAR-CAKE-SI': 3,
   'SCONE-HTSI': 7,
   'CAR-CAKE-TG': 1,
   'PAC-TG': 4,
   'CC-BROW-TG': 1,
   ' WM-MUF-TG': 2,
   'RASP-FIN-SI': 1,
   'SNICK-BALL-SI': 2,
   'BW-MUF-TG': 1,
   ' WM-MUF-SI': 2,
   'OATCOOK-TG': 2,
   'CIN-BUN-TG': 6,
   'BW-MUF-SI': 2,
   'SNICK-BALL-TG': 6,
   'CROIS-TG': 2,
   'ANZAC-SI': 3,
   'CIN-BUN-SI': 1,
   'ALCROIS-SI': 1,
   'ALCROIS-TG': 4},
  'BREAKFAST': {'HC-CROIS-H-SI': 1,
   'POR-HON-SI': 1,
   'POR-CHH-TG': 2,
   'TOAST-TG': 2,
   'HC-CROIS-NH-TG': 1,
   'SROLL-TG': 2,
   'BERPAR-TG': 2,
   'HC-CROIS-H-TG': 1},
  'LUNCH': {'SOUP-SI': 1,
   'REUB-NH-TG': 1,
   'FAL-WRAP-NH-TG': 1,
   'CHIC-POCK-NH-TG': 1,
   'HCTOAST-H-TG': 1}},
 '11/02/2017 00:00:00': {'BAKED GOODS': {'ALM-FIN-T

In [292]:
currentyearsales

,11/01/2018 00:00:00,11/02/2018 00:00:00,11/03/2018 00:00:00,11/04/2018 00:00:00,11/05/2018 00:00:00,11/06/2018 00:00:00,11/07/2018 00:00:00,11/08/2018 00:00:00,11/09/2018 00:00:00,11/10/2018 00:00:00,...,01/21/2019 00:00:00,01/22/2019 00:00:00,01/23/2019 00:00:00,01/24/2019 00:00:00,01/25/2019 00:00:00,01/26/2019 00:00:00,01/27/2019 00:00:00,01/28/2019 00:00:00,01/29/2019 00:00:00,01/30/2019 00:00:00
BAKED GOODS,"{'ALM-FIN-TG': 3, 'CUST-TART-TG': 3, 'CRUFF-TG...","{'ALM-FIN-TG': 6, 'CUST-TART-TG': 4, 'CRUFF-TG...","{'ALM-FIN-TG': 7, 'SCONE-NHTG': 6, 'CUST-TART-...","{'ALM-FIN-TG': 6, 'SCONE-NHTG': 2, 'CUST-TART-...","{'CRUFF-SI': 3, 'ALM-FIN-TG': 4, 'SCONE-NHTG':...","{'CRUFF-SI': 2, 'ALM-FIN-TG': 4, 'SCONE-NHTG':...","{'CRUFF-SI': 1, 'ALM-FIN-TG': 5, 'SCONE-NHTG':...","{'CRUFF-SI': 1, 'ALM-FIN-TG': 4, 'SCONE-NHTG':...","{'CRUFF-SI': 3, 'ALM-FIN-TG': 6, 'SCONE-NHTG':...","{'ALM-FIN-TG': 1, 'SCONE-NHTG': 5, 'CUST-TART-...",...,"{'SCONE-HTG': 7, 'CIN-BUN-TG': 4, 'PAIN-CHOC-T...","{'SCONE-HTG': 6, 'CIN-BUN-TG': 6, 'PAIN-CHOC-S...","{'SCONE-HTG': 5, 'CIN-BUN-TG': 4, 'PAIN-CHOC-S...","{'SCONE-HTG': 12, 'CIN-BUN-TG': 4, 'OATCOOK-SI...","{'SCONE-HTG': 14, 'CIN-BUN-TG': 6, 'PAIN-CHOC-...","{'SCONE-HTG': 1, 'CIN-BUN-TG': 5, 'PAIN-CHOC-S...","{'SCONE-HTG': 4, 'CIN-BUN-TG': 4, 'PAIN-CHOC-T...","{'SCONE-HTG': 2, 'CIN-BUN-TG': 3, 'PAIN-CHOC-S...","{'SCONE-HTG': 11, 'CIN-BUN-TG': 3, 'PAIN-CHOC-...","{'SCONE-HTG': 4, 'CIN-BUN-TG': 4, 'PAIN-CHOC-S..."
BREAKFAST,"{'SROLL-SI': 1, 'HCCROIS-H-TG': 1, 'CHIA-POT-S...","{'SROLL-SI': 1, 'POR-CHH-SI': 4, 'HCCROIS-H-TG...","{'SROLL-SI': 5, 'SROLL-TG': 1, 'BERPAR-TG': 1,...","{'SROLL-SI': 2, 'BERPAR-SI': 2, 'HCCROIS-H-SI'...","{'SROLL-SI': 3, 'POR-CHH-TG': 4, 'TOAST-TG': 1...","{'SROLL-SI': 2, 'POR-CHH-TG': 4, 'TOAST-TG': 2...","{'POR-CHH-TG': 3, 'SROLL-TG': 7, 'POR-CHH-SI':...","{'SROLL-SI': 1, 'POR-HON-SI': 1, 'POR-CHH-TG':...","{'SROLL-SI': 4, 'POR-CHH-TG': 7, 'SROLL-TG': 4...","{'SROLL-SI': 3, 'SROLL-TG': 2, 'HCCROIS-H-TG':...",...,"{'TOAST-TG': 3, 'SROLL-TG': 5, 'POR-CHH-TG': 8...","{'TOAST-SI': 1, 'TOAST-TG': 2, 'SROLL-TG': 1, ...","{'TOAST-TG': 3, 'SROLL-TG': 3, ' POR-CHH-SI': ...","{'TOAST-SI': 1, 'TOAST-TG': 5, 'SROLL-TG': 6, ...","{'TOAST-SI': 1, 'TOAST-TG': 2, 'SROLL-TG': 6, ...","{'SROLL-TG': 1, 'HCCROIS-H-TG': 1, 'BERPAR-SI'...","{'SROLL-TG': 1, 'HCCROIS-H-TG': 1, 'HCCROIS-NH...","{'TOAST-SI': 2, 'SROLL-TG': 1, 'POR-CHH-TG': 5...","{'TOAST-TG': 2, 'SROLL-TG': 1, 'POR-CHH-TG': 1...","{'TOAST-SI': 1, 'TOAST-TG': 3, 'SROLL-TG': 2, ..."
LUNCH,"{'P-HAM-NH-TG': 1, 'CAPRESE-NH-TG': 2, 'HCTOAS...","{'P-HAM-NH-TG': 1, 'CAPRESE-NH-TG': 2, 'HCTOAS...","{'CAPRESE-NH-TG': 1, 'CHICKBAC-WRAP-NH-TG': 1,...","{'HCTOAST-H-SI': 1, 'CAPRESE-H-SI': 1}","{'P-HAM-NH-TG': 2, 'SOUP-SI': 1, 'SCBW-H-SI': ...","{'VGN-WRAP-NH-TG': 1, 'CAPRESE-NH-TG': 1, 'SOU...","{'P-HAM-NH-TG': 1, 'CAPRESE-NH-TG': 1, 'SOUP-T...","{'SHCT-H-SI': 1, 'SOUP-SI': 1, 'STUNA-H-SI': 1...","{'SHCT-H-SI': 2, 'SCAP-NH-TG': 1, 'VGN-WRAP-NH...","{'P-HAM-NH-TG': 1, 'VGN-WRAP-NH-TG': 1, 'HCTOA...",...,"{'TUR-SAN-NH-TG': 2, 'CHICK-SER-NH-SI': 1, 'VG...","{'VGN-WRAP-NH-TG': 1, 'HCTOAST-H-TG': 1, 'HCTO...","{'VGN-WRAP-NH-SI': 1, 'CHICKBAC-WRAP-NH-TG': 1...","{'CAPRESE-H-TG': 1, 'HCTOAST-H-TG': 1, 'HCTOAS...","{'TUR-SAN-NH-TG': 2, 'HCTOAST-H-TG': 1, 'CHICK...","{'TUR-SAN-NH-TG': 1, 'HCTOAST-H-TG': 2, 'TUR-S...",{},"{'TUR-SAN-H-SI': 2, 'CHICKBAC-WRAP-NH-TG': 1, ...","{'CAPRESE-H-TG': 1, 'VGN-WRAP-NH-TG': 1, 'HCTO...","{'TUR-SAN-NH-TG': 1, 'HCTOAST-H-TG': 2, 'CAPRE..."


In [258]:
prevyearsales = pd.DataFrame(prevyearsales)
currentyearsales = pd.DataFrame(currentyearsales)

In [262]:
def sum_sales_by_category(df):
    salesbycategory = {}
    for column in df.columns:
        salesbycategory[column] = {}
        salesbycategory[column]['dayofweek'] = pd.Timestamp(pd.to_datetime(column)).dayofweek

        for index in dailysales.index:
            item = df.loc[index, column]
            total = sum(item[k] for k in item.keys())
            salesbycategory[column][index] = total
    return salesbycategory

In [263]:
p_salesbycategory = sum_sales_by_category(prevyearsales)
c_salesbycategory = sum_sales_by_category(currentyearsales)

In [264]:
p_salesbycategory

{'11/01/2017 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 88,
  'BREAKFAST': 12,
  'LUNCH': 5},
 '11/02/2017 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 94,
  'BREAKFAST': 12,
  'LUNCH': 12},
 '11/03/2017 00:00:00': {'dayofweek': 4,
  'BAKED GOODS': 86,
  'BREAKFAST': 12,
  'LUNCH': 14},
 '11/04/2017 00:00:00': {'dayofweek': 5,
  'BAKED GOODS': 82,
  'BREAKFAST': 9,
  'LUNCH': 6},
 '11/05/2017 00:00:00': {'dayofweek': 6,
  'BAKED GOODS': 42,
  'BREAKFAST': 5,
  'LUNCH': 6},
 '11/06/2017 00:00:00': {'dayofweek': 0,
  'BAKED GOODS': 61,
  'BREAKFAST': 7,
  'LUNCH': 6},
 '11/07/2017 00:00:00': {'dayofweek': 1,
  'BAKED GOODS': 84,
  'BREAKFAST': 11,
  'LUNCH': 11},
 '11/08/2017 00:00:00': {'dayofweek': 2,
  'BAKED GOODS': 83,
  'BREAKFAST': 13,
  'LUNCH': 8},
 '11/09/2017 00:00:00': {'dayofweek': 3,
  'BAKED GOODS': 73,
  'BREAKFAST': 10,
  'LUNCH': 10},
 '11/10/2017 00:00:00': {'dayofweek': 4,
  'BAKED GOODS': 103,
  'BREAKFAST': 20,
  'LUNCH': 10},
 '11/11/2017 00:00:00': {'dayofweek':

In [265]:
p_salesbycategory = pd.DataFrame(p_salesbycategory)
c_salesbycategory = pd.DataFrame(c_salesbycategory)

In [266]:
p_salesbycategory

,11/01/2017 00:00:00,11/02/2017 00:00:00,11/03/2017 00:00:00,11/04/2017 00:00:00,11/05/2017 00:00:00,11/06/2017 00:00:00,11/07/2017 00:00:00,11/08/2017 00:00:00,11/09/2017 00:00:00,11/10/2017 00:00:00,...,01/21/2018 00:00:00,01/22/2018 00:00:00,01/23/2018 00:00:00,01/24/2018 00:00:00,01/25/2018 00:00:00,01/26/2018 00:00:00,01/27/2018 00:00:00,01/28/2018 00:00:00,01/29/2018 00:00:00,01/30/2018 00:00:00
dayofweek,2,3,4,5,6,0,1,2,3,4,...,6,0,1,2,3,4,5,6,0,1
BAKED GOODS,88,94,86,82,42,61,84,83,73,103,...,33,55,67,68,74,88,62,39,48,61
BREAKFAST,12,12,12,9,5,7,11,13,10,20,...,5,12,13,10,11,13,9,4,11,13
LUNCH,5,12,14,6,6,6,11,8,10,10,...,0,9,9,11,9,10,3,1,7,10


In [267]:
c_salesbycategory

,11/01/2018 00:00:00,11/02/2018 00:00:00,11/03/2018 00:00:00,11/04/2018 00:00:00,11/05/2018 00:00:00,11/06/2018 00:00:00,11/07/2018 00:00:00,11/08/2018 00:00:00,11/09/2018 00:00:00,11/10/2018 00:00:00,...,01/21/2019 00:00:00,01/22/2019 00:00:00,01/23/2019 00:00:00,01/24/2019 00:00:00,01/25/2019 00:00:00,01/26/2019 00:00:00,01/27/2019 00:00:00,01/28/2019 00:00:00,01/29/2019 00:00:00,01/30/2019 00:00:00
dayofweek,3,4,5,6,0,1,2,3,4,5,...,0,1,2,3,4,5,6,0,1,2
BAKED GOODS,94,106,73,64,69,94,80,82,106,89,...,64,84,73,81,108,58,57,63,76,79
BREAKFAST,28,32,11,6,22,31,22,29,32,13,...,26,17,22,31,20,6,8,18,13,21
LUNCH,15,18,11,2,10,15,13,13,20,7,...,10,7,10,13,14,10,0,11,10,16


In [269]:
def find_average_sales_by_day(df):
    bakedgoods = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    breakfast = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    lunch = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    daysinamonth = {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0}
    averages = {}
    
    for date in df.columns:
        for category in df.index:
            if category == 'dayofweek':
                week = df.loc[category, date]
                daysinamonth[str(week)] += 1
            elif category == 'BAKED GOODS':
                bakedgoods[str(week)] += df.loc[category, date]
            elif category == 'BREAKFAST':
                breakfast[str(week)] += df.loc[category, date]
            elif category == 'LUNCH':
                lunch[str(week)] += df.loc[category, date]
                
    totalsalesbycategory = {'bakedgoods':bakedgoods, 'breakfast':breakfast, 'lunch':lunch}
    
    for category, total in totalsalesbycategory.items():
        averages[category] = {}
        for day, sales in total.items():
            averages[category][day] = format(sales/daysinamonth[day], '.1f')

    return averages

In [270]:
p_average = find_average_sales_by_day(p_salesbycategory)
c_average = find_average_sales_by_day(c_salesbycategory)

In [273]:
p_average = pd.DataFrame(p_average)
p_average

,bakedgoods,breakfast,lunch
0,50.7,10.3,6.4
1,60.4,11.5,7.8
2,70.5,11.1,8.5
3,78.8,12.4,8.6
4,88.6,15.5,8.6
5,79.3,8.5,6.8
6,49.8,5.1,4.3


In [274]:
c_average = pd.DataFrame(c_average)
c_average

,bakedgoods,breakfast,lunch
0,75.6,20.6,12.6
1,65.4,17.6,10.8
2,73.7,21.2,14.3
3,88.2,23.2,14.0
4,98.3,26.2,17.5
5,83.1,11.3,11.5
6,66.9,8.8,4.0


In [ ]:
# % increase = Increase ÷ Original Number × 100.

In [283]:
from decimal import *

In [288]:
percentagechange = {}
for index in c_average.index:
    percentagechange[index] = {}
    for category in c_average.columns:
        current = c_average.loc[index, category]
        prev = p_average.loc[index, category]
        difference = Decimal(current) - Decimal(prev)
        change = (Decimal(current) - Decimal(prev)) / Decimal(current) * Decimal(100)
        percentagechange[index][category] = format(change, '.1f')

In [290]:
percentagechange = pd.DataFrame(percentagechange)

In [291]:
percentagechange

,0,1,2,3,4,5,6
bakedgoods,32.9,7.6,4.3,10.7,9.9,4.6,25.6
breakfast,50.0,34.7,47.6,46.6,40.8,24.8,42.0
lunch,49.2,27.8,40.6,38.6,50.9,40.9,-7.5
